# Import the libraries and the data

In [1]:
from PIL import Image, ImageDraw, ImageFont
import numpy as np
import pandas as pd
import folium
from folium import plugins
import matplotlib.pyplot as plt
import matplotlib.colors
from matplotlib.colors import LinearSegmentedColormap, rgb_to_hsv, hsv_to_rgb
import scipy.ndimage.filters
import time
import datetime as dt
import os.path
import io

from selenium import webdriver

%matplotlib inline

In [2]:
data = pd.read_csv('df_10_10_19.csv')

In [3]:
data

,station_id,num_bikes_available,num_bikes_available_types.mechanical,num_bikes_available_types.ebike,num_docks_available,is_installed,is_renting,is_returning,last_reported,is_charging_station,...,capacity,year,month,day,day_of_week,hour,minute,second,IsWeekday,TimeSlice
0,101,0,0,0,24,1,1,1,1.570666e+09,True,...,24.0,2019,10,10,3,0,0,0,True,0
1,288,4,1,3,27,1,1,1,1.570666e+09,True,...,32.0,2019,10,10,3,0,0,3,True,0
2,1,28,28,0,1,1,1,1,1.570666e+09,True,...,30.0,2019,10,10,3,0,2,11,True,0
3,2,25,25,0,1,1,1,1,1.570666e+09,True,...,27.0,2019,10,10,3,0,2,47,True,0
4,3,20,20,0,7,1,1,1,1.570666e+09,True,...,27.0,2019,10,10,3,0,0,59,True,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
25336,427,20,20,0,0,1,1,1,1.570751e+09,True,...,20.0,2019,10,10,3,23,42,31,True,71
25337,492,4,4,0,20,1,1,1,1.570751e+09,True,...,24.0,2019,10,10,3,23,43,0,True,71
25338,494,0,0,0,23,1,1,1,1.570751e+09,True,...,24.0,2019,10,10,3,23,45,6,True,71
25339,495,1,1,0,21,1,1,1,1.570751e+09,True,...,24.0,2019,10,10,3,23,43,34,True,71


# Define the function to create maps

In [4]:
def availability_map(df, time_slice):
    
    
    Barcelona = [41.3887901, 2.1589899]
    mapa = folium.Map(location = Barcelona, 
                   zoom_start = 12.5, 
                   tiles = "CartoDB dark_matter")
    
    df['percentatge'] = round((df['num_bikes_available']/(df['num_bikes_available']+df['num_docks_available']))*100)
    
    choices = ['A', 'B', 'C', 'D', 'E']
    conditions = [(df['num_bikes_available']==0), (df['num_bikes_available'].between(1, 5)), ((df['num_bikes_available']>5) & (df['num_docks_available']>5)), (df['num_docks_available'].between(1, 5)), (df['num_docks_available']==0)]
    df['index_order'] = np.select(conditions, choices, default=np.nan)
    
    df = df[df['TimeSlice'] == time_slice]
    
    
    colordict = {'A': 'red', 'B': 'orange', 'C': 'yellow', 'D': 'lightblue', 'E':'green', 'nan':'pink'}


    for lat, lon, capacity, av_bikes, traffic_q, stations, percent in zip(df['lat'], df['lon'], df['capacity'], df['num_bikes_available'], df['index_order'], df['station_id'], df['percentatge']):
        folium.CircleMarker([lat, lon], radius=5, popup=('Station: ' + str(stations).capitalize() + '<br>'
                 'Bikes available: ' + str(av_bikes) + '<br>'
                 '% of availability: ' + str(percent) +'%'
                ), color='b', fill_color=colordict[traffic_q], fill_opacity=0.7).add_to(mapa)

    return(mapa)

Let's test the function

In [5]:
availability_map(data, 20)

In [6]:
def go_arrivals_frame_2(dataframe, i, timeslice):
    
    """
    Function to take screenshots from several images over time
    input: dataframe-> dataframe
           i -> counter
           time_slice -> time slice
           
    Note! You must download chromedriver (compatible with your chrome version) and paste it in the same path 
    
    """
    import os
    # create the map object calling the function that we've defined previously
    my_frame = availability_map(dataframe,timeslice)
    
    # Define the path in which you want to save the images
    delay=5
    fn="map_{:0}".format(i)
    DIR = '/Users/nuriaromeroherreros/Desktop/allWomen/FINAL_PROJECT/IMAGES_2/'
    os.chdir(DIR)

    
    # Name of the images, sequentially
    f = DIR + fn + '.html'
    print(f)
    
    # save the image in .html format
    my_frame.save(f)
    
    

    # instantiate the webdriver
    browser = webdriver.Chrome(executable_path='/Users/nuriaromeroherreros/Desktop/chromedriver')
    
    # open the .html file
    browser.get('file://'+f)
    # file:///Users/nuriaromeroherreros/Desktop/allWomen/FINAL%20PROJECT/IMAGES/map_0.html
    
    #Give the map tiles some time to load
    time.sleep(delay)
    
    # Take a screenshot and save it in .png extension
    browser.save_screenshot(f+'.png')
    browser.quit()

In [7]:
# As first stepm define the timeslice that you want to animate
np.arange(0, 2)

array([0, 1])

In [8]:
# Generate frames by creating a look

for i, time_slice in enumerate(np.arange(0, 71)):
    print(i, time_slice)
    go_arrivals_frame_2(data,i, time_slice)

0 0
/Users/nuriaromeroherreros/Desktop/allWomen/FINAL_PROJECT/IMAGES_2/map_0.html
1 1
/Users/nuriaromeroherreros/Desktop/allWomen/FINAL_PROJECT/IMAGES_2/map_1.html
2 2
/Users/nuriaromeroherreros/Desktop/allWomen/FINAL_PROJECT/IMAGES_2/map_2.html
3 3
/Users/nuriaromeroherreros/Desktop/allWomen/FINAL_PROJECT/IMAGES_2/map_3.html
4 4
/Users/nuriaromeroherreros/Desktop/allWomen/FINAL_PROJECT/IMAGES_2/map_4.html
5 5
/Users/nuriaromeroherreros/Desktop/allWomen/FINAL_PROJECT/IMAGES_2/map_5.html
6 6
/Users/nuriaromeroherreros/Desktop/allWomen/FINAL_PROJECT/IMAGES_2/map_6.html
7 7
/Users/nuriaromeroherreros/Desktop/allWomen/FINAL_PROJECT/IMAGES_2/map_7.html
8 8
/Users/nuriaromeroherreros/Desktop/allWomen/FINAL_PROJECT/IMAGES_2/map_8.html
9 9
/Users/nuriaromeroherreros/Desktop/allWomen/FINAL_PROJECT/IMAGES_2/map_9.html
10 10
/Users/nuriaromeroherreros/Desktop/allWomen/FINAL_PROJECT/IMAGES_2/map_10.html
11 11
/Users/nuriaromeroherreros/Desktop/allWomen/FINAL_PROJECT/IMAGES_2/map_11.html
12 12
/Use

# Save the frames into .gif extension

In [38]:
import os
import imageio

def save_gifs(png_dir, gif_dir):
    """
    
    """
    images = []
    for file_name in os.listdir(png_dir):
        if file_name.endswith('.png'):
            file_path = os.path.join(png_dir, file_name)
            images.append(imageio.imread(file_path))
            
    imageio.mimsave(gif_dir, images)

In [39]:
png_dir = '../images/'
gif_dir = '../images/Maps.gif'

save_gifs(png_dir, gif_dir)